# 30-Day Scheduler Sync (Refactored)

This notebook syncs event data from a Google Sheet to a Google Calendar. It leverage centralized utilities for consistency.

## Step 1: Setup and Authentication

In [1]:
import os
import sys
import pandas as pd
import datetime

# Add src to path so we can import our modules
sys.path.append('src')

from utils_calendar_general import get_google_services, create_calendar_event
from sync_scheduler import prepare_event_body

# --- Configuration ---
SPREADSHEET_ID = '10Z993MrZHH0Da_pXEFZoo0MBdxKhf619fZSuuvaAdlQ'
SIGNUP_SHEET = 'Signup'
CONTACT_SHEET = 'Teacher Contact'
CALENDAR_ID = 'b45a2d5121fed950d815cfa167dd4b3a6aa74c5d62fea928702e6f4300d96545@group.calendar.google.com'
TEMPLATE_FILE = '_calendar_event_template.jsonc'
CREATED_EVENTS_CSV = 'logs/created_events.csv'
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets.readonly', 
    'https://www.googleapis.com/auth/calendar.events'
]

# Credentials Paths
TOKEN_PATH = '.credentials/token.json'
CREDS_PATH = '.credentials/credentials.json'

sheets_service, calendar_service = get_google_services(TOKEN_PATH, CREDS_PATH, SCOPES)
print("Services initialized.")

Services initialized.


### Find and Delete Events

Search for events on the calendar by search string

In [4]:
# Assuming 'service' is the authenticated Google Calendar API service object
search_query = "45"
now = datetime.datetime.utcnow().isoformat() + 'Z'

events_result = calendar_service.events().list(
    calendarId=CALENDAR_ID,
    timeMin=now,
    q=search_query,
    singleEvents=True,
    orderBy='startTime'
).execute()

events = events_result.get('items', [])
event_data = [{"summary": event.get("summary"), "id": event.get("id")} for event in events]

for event in event_data:
    print(event["summary"])



45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute Guided Session: Stephen Holsenbeck
45-Minute 

/var/folders/s8/yv0hmbdj5pq2g88v8qs7v82c0000gn/T/ipykernel_8413/2886857036.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'


Delete calendar events found in previous chunk

In [ ]:
for event in event_data:
    calendar_service.events().delete(calendarId=CALENDAR_ID, eventId=event['id']).execute()
    print(f"Deleted: {event['summary']}")

df_log = pd.read_csv(CREATED_EVENTS_CSV)
deleted_ids = [event['id'] for event in event_data]
df_log = df_log[~df_log['ID'].isin(deleted_ids)]
df_log.to_csv(CREATED_EVENTS_CSV, index=False)


Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Session: Stephen Holsenbeck
Deleted: 45-Minute Guided Se

KeyError: 'id'

## Step 2: Fetch Event Data from Google Sheets

In [6]:
from google_sheets_data import fetch_google_sheets_data

# Fetch data using the external module
df_pending, teacher_map = fetch_google_sheets_data(
    sheets_service=sheets_service,
    spreadsheet_id=SPREADSHEET_ID,
    signup_sheet=SIGNUP_SHEET,
    contact_sheet=CONTACT_SHEET,
    template_file=TEMPLATE_FILE
)
display(df_pending.head())

Loaded 14 teacher contacts.
Found 61 events in Google Sheets.


,Summary,Begin,Teacher,Contact,Date,Day,Start,End,Duration
0,45-Minute Guided Session: Stephen Holsenbeck,2026-02-01T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-01,Sunday,07:00,07:45,45
1,45-Minute Guided Session: Stephen Holsenbeck,2026-02-02T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-02,Monday,07:00,07:45,45
2,45-Minute Guided Session: Stephen Holsenbeck,2026-02-03T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-03,Tuesday,07:00,07:45,45
3,10-Minute Guided Session: Brad Constable,2026-02-03T08:00:00,Brad Constable,"{'First Name': 'Brad', 'Last Name': 'Constable...",2026-02-03,Tuesday,08:00,08:10,10
4,10-Minute Guided Session: Eileen Knott,2026-02-03T20:00:00,Eileen Knott,"{'First Name': 'Eileen', 'Last Name': 'Knott',...",2026-02-03,Tuesday,20:00,20:10,10


## Step 3: Overlap Detection

In [7]:
from overlap_detection import check_overlaps, update_created_events_csv

UPDATE_CREATED_EVENTS = True # Set to True to refresh from Calendar

if UPDATE_CREATED_EVENTS:
    update_created_events_csv(calendar_service, CALENDAR_ID, CREATED_EVENTS_CSV)

if os.path.exists(CREATED_EVENTS_CSV):
    df_created = pd.read_csv(CREATED_EVENTS_CSV)
    print(f"Loaded {len(df_created)} already created events.")
    
    # Mark overlaps
    df_pending['is_overlap'] = check_overlaps(df_pending, df_created)
    
    overlaps = df_pending[df_pending['is_overlap'] == True]
    print(f"Detected {len(overlaps)} overlapping events.")
    display(overlaps.head())
else:
    df_pending['is_overlap'] = False
    print("No created_events.csv found. Starting fresh.")

Syncing logs/created_events.csv with Google Calendar...
Successfully synced 250 events to logs/created_events.csv.
Loaded 250 already created events.
Detected 23 overlapping events.


,Summary,Begin,Teacher,Contact,Date,Day,Start,End,Duration,is_overlap
0,45-Minute Guided Session: Stephen Holsenbeck,2026-02-01T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-01,Sunday,07:00,07:45,45,True
4,10-Minute Guided Session: Eileen Knott,2026-02-03T20:00:00,Eileen Knott,"{'First Name': 'Eileen', 'Last Name': 'Knott',...",2026-02-03,Tuesday,20:00,20:10,10,True
8,10-Minute Guided Session: Noah Binder,2026-02-05T15:00:00,Noah Binder,"{'First Name': 'Noah', 'Last Name': 'Binder', ...",2026-02-05,Thursday,15:00,15:10,10,True
11,10-Minute Guided Session: Rob Thompson,2026-02-06T12:00:00,Rob Thompson,"{'First Name': 'Rob', 'Last Name': 'Thompson',...",2026-02-06,Friday,12:00,12:10,10,True
12,10-Minute Guided Session: Noah Binder,2026-02-06T15:00:00,Noah Binder,"{'First Name': 'Noah', 'Last Name': 'Binder', ...",2026-02-06,Friday,15:00,15:10,10,True


## Step 4: User Review and Pruning

In [8]:
print("Review the overlaps above.")
confirm = input("Remove overlapping events from the list to be created? (y/n): ")

if confirm.lower() == 'y':
    df_to_create = df_pending[df_pending['is_overlap'] == False].copy()
    print(f"Pruned {len(df_pending) - len(df_to_create)} events. {len(df_to_create)} remaining.")
else:
    df_to_create = df_pending.copy()
    print(f"Proceeding with all {len(df_to_create)} events.")
display(df_to_create.head())

Review the overlaps above.
Pruned 23 events. 38 remaining.


,Summary,Begin,Teacher,Contact,Date,Day,Start,End,Duration,is_overlap
1,45-Minute Guided Session: Stephen Holsenbeck,2026-02-02T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-02,Monday,07:00,07:45,45,False
2,45-Minute Guided Session: Stephen Holsenbeck,2026-02-03T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-03,Tuesday,07:00,07:45,45,False
3,10-Minute Guided Session: Brad Constable,2026-02-03T08:00:00,Brad Constable,"{'First Name': 'Brad', 'Last Name': 'Constable...",2026-02-03,Tuesday,08:00,08:10,10,False
5,10-Minute Guided Session: Rob Thompson,2026-02-03T21:00:00,Rob Thompson,"{'First Name': 'Rob', 'Last Name': 'Thompson',...",2026-02-03,Tuesday,21:00,21:10,10,False
6,45-Minute Guided Session: Stephen Holsenbeck,2026-02-04T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-04,Wednesday,07:00,07:45,45,False


## Step 5: Preview and Create Events

In [9]:
from sync_scheduler import create_scheduled_events

# Prepare the events from the dataframe
events_to_create = []
template_text = open(TEMPLATE_FILE, 'r').read()

for idx, row in df_to_create.iterrows():
    events_to_create.append(prepare_event_body(row, template_text))

print(f"Events ready for creation: {len(events_to_create)}")
run_confirm = input("Proceed with event creation? (y/n): ")

if run_confirm.lower() == 'y':
    # This now uses the shared logic in src/sync_scheduler.py 
    # which includes the break-on-failure and debug prints!
    create_scheduled_events(calendar_service, CALENDAR_ID, events_to_create, CREATED_EVENTS_CSV)
else:
    print("Creation cancelled.")

Events ready for creation: 38
Created event: https://www.google.com/calendar/event?eid=NDZ0YjR0NHRwcTRiaTZpaDJoaGlrZWN1Y2cgYjQ1YTJkNTEyMWZlZDk1MGQ4MTVjZmExNjdkZDRiM2E2YWE3NGM1ZDYyZmVhOTI4NzAyZTZmNDMwMGQ5NjU0NUBn
Created event: https://www.google.com/calendar/event?eid=cGhmNGJhbmJsMmhqZWNmZnA0amxiOWRtbzAgYjQ1YTJkNTEyMWZlZDk1MGQ4MTVjZmExNjdkZDRiM2E2YWE3NGM1ZDYyZmVhOTI4NzAyZTZmNDMwMGQ5NjU0NUBn
Created event: https://www.google.com/calendar/event?eid=amh0YW5tOWtoaDNoMHA0NDhpYzFhcGdhOGcgYjQ1YTJkNTEyMWZlZDk1MGQ4MTVjZmExNjdkZDRiM2E2YWE3NGM1ZDYyZmVhOTI4NzAyZTZmNDMwMGQ5NjU0NUBn
Created event: https://www.google.com/calendar/event?eid=ampsM2NpMDQ4b3ZoN3B1ZmVqajk0OWxwZWMgYjQ1YTJkNTEyMWZlZDk1MGQ4MTVjZmExNjdkZDRiM2E2YWE3NGM1ZDYyZmVhOTI4NzAyZTZmNDMwMGQ5NjU0NUBn
Created event: https://www.google.com/calendar/event?eid=amU5bzk3bzNtdW5kMWkxdm5uZDdidnVpZ28gYjQ1YTJkNTEyMWZlZDk1MGQ4MTVjZmExNjdkZDRiM2E2YWE3NGM1ZDYyZmVhOTI4NzAyZTZmNDMwMGQ5NjU0NUBn
Created event: https://www.google.com/calendar/event?eid=dHN

## Step 6: Delete Events (Cleanup)

In [ ]:
from utils_calendar_general import delete_events_from_csv

# To delete events, uncomment and run the line below:
# delete_events_from_csv(CREATED_EVENTS_CSV, calendar_service, CALENDAR_ID)